In [162]:
#pip install --no-cache-dir psycopg2-binary

In [19]:
import psycopg2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [241]:
try:
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    print("Connected to Redshift successfully!")
except Exception as e:
    print("Connection failed: ", e)

# Don't forget to close the connection when done
# conn.close()

Connected to Redshift successfully!


In [142]:
query = "SELECT * FROM fact_sensor_readings;"

In [148]:
#conn.close()

In [32]:
q_sensors = 'select * from dim_sensors;'
q_time = 'select * from dim_time;'
q_equipments = 'select * from dim_equipments;'
q_plants = 'select * from dim_plants;'

In [36]:
df_time = pd.read_sql(q_time, conn)

In [242]:
df_sensors = pd.read_sql(q_sensors, conn)
#df_sensors.name = df_sensors.name.str.rstrip('\t')

In [244]:
df_sensors.name.unique()

array(['Thermocouple', 'Vibration Sensor', 'Temperature Sensor\t',
       'Voltage Sensor', 'Coolant temperature', 'Pressure Sensor',
       'Oil Temperature Sensor\t', 'Moisture Sensor\t'], dtype=object)

In [38]:
df_equipments = pd.read_sql(q_equipments, conn)

In [39]:
df_plants = pd.read_sql(q_plants, conn)

In [143]:
df = pd.read_sql(query, conn)

In [145]:
df = df.merge(
    df_time, on = 'time_id', how='left'
).merge(df_sensors.drop('equipment_id', axis = 1), on = 'sensor_id', how='left'
       ).rename(
    {'name':'sensor_name'}, axis = 1
).merge(df_equipments.drop('plant_id', axis = 1), on = 'equipment_id', how='left').rename(
    {'name':'equipment_name', 'type':'equipment_type'}, axis = 1
).merge(df_plants, on='plant_id', how='left').rename({'name':'plant_name'},axis=1)
df.date_value = pd.to_datetime(df.date_value)

In [252]:
df.head()

,reading_id,sensor_id,time_id,value,equipment_id,plant_id,processed_at,date_value,year,month,...,hour,sensor_name,unit,equipment_name,equipment_type,plant_name,region,latitude,longitude,mean_temp
0,1,3000001,1672617600,1133.09,2000001,1001,1743890400,2023-01-02,2023,1,...,0,Thermocouple,Celsius,Gas turbine,Turbine,Baja Norte Plant,Northwest,32.52,-117.03,1100.447969
1,4,3000317,1687176000,986.02,2000114,1008,1743890400,2023-06-19,2023,6,...,12,Temperature Sensor,Celsius,Gas generator,Generator,Yucatan Power Station,Southwest,20.97,-89.62,950.383903
2,5,3000001,1672704000,1114.60,2000001,1001,1743890400,2023-01-03,2023,1,...,0,Thermocouple,Celsius,Gas turbine,Turbine,Baja Norte Plant,Northwest,32.52,-117.03,1100.447969
3,8,3000317,1687262400,1069.72,2000114,1008,1743890400,2023-06-20,2023,6,...,12,Temperature Sensor,Celsius,Gas generator,Generator,Yucatan Power Station,Southwest,20.97,-89.62,950.383903
4,9,3000001,1672790400,852.07,2000001,1001,1743890400,2023-01-04,2023,1,...,0,Thermocouple,Celsius,Gas turbine,Turbine,Baja Norte Plant,Northwest,32.52,-117.03,1100.447969


z-score

In [251]:
df['mean_temp'] = df.groupby(['equipment_name','sensor_name'])['value'].transform(lambda x: x.mean())
df['std_temp'] = df.groupby(['equipment_name','sensor_name'])['value'].transform(lambda x: x.std())

In [255]:
df['z_score'] = (df['value'] - df['mean_temp']) / df['std_temp']

In [256]:
# Flag anomalies (+ 3 standard deviations)
df['anomaly'] = df['z_score'].abs() > 3

In [257]:
df = df.drop(columns=["mean_temp", "std_temp", "z_score"])

In [269]:
df.groupby(['equipment_name','sensor_name']).anomaly.value_counts(1).unstack().style.format('{:.2%}')

---